# Connect to Extrernal source of database

In [1]:
#@title After exciting this cell you Have to connect it with You Google Drive. 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@title Define the Hyper-Parameters
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.random import normal

image_height = 256 #@param {type:"integer"} 
image_width = 256 #@param {type:"integer"}
filters = 64 #@param {type:"integer"}
output_stride = 16 #@param {type:"integer"}
height_output = image_height // output_stride 
width_output = image_width // output_stride
batch_size = 32 #@param {type:"integer"}
latent_dim = 100 #@param {type:"integer"}

display_noise = normal(shape=[16, latent_dim], mean=0, stddev=1)
w_init = tf.initializers.glorot_uniform()

In [ ]:
#@title Now we will check the GPU
!nvidia-smi

In [4]:
#@title Preprocess the data and define model paths 
from tensorflow.image import resize,decode_png
from tensorflow.data.experimental import AUTOTUNE
from tensorflow.io import read_file
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 
from glob import glob
import numpy as np
import cv2,os

Main_folder_path = "old/new_idea_test" #@param {type:"string"}
os.chdir(os.path.join("/content/drive/My Drive",Main_folder_path))

Augmentation_flag = False #@param {type: "boolean"} 
Augmentation_folder = "path" #@param {type:"string"}
images_path = "ll" #@param {type:"string"}

############################################################# Augmentation part
def Augment_images(image_file):
  datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.1,	horizontal_flip=True, vertical_flip=True, fill_mode="nearest")

  aug_img = load_img(os.path.join(images_path,image_file)) 
  aug_img = img_to_array(aug_img)
  aug_img = aug_img.reshape((1,) + x.shape) 

  i = 0
  for batch in datagen.flow(x, batch_size=1,save_to_dir='new', save_prefix='image_file', save_format='jpg'):
      i += 1
      if i > 50: break

if Augmentation_flag:
  for image_name in os.listdir("ll"):   
      print ("Augmenting... Image name: "+image_name)
      Augment_images(i)

############################################################# Preprocessing
def data_load(images_path):
    img = resize(decode_png(read_file(images_path)), size=[image_height, image_width])[..., :3]
    img /= 127.5
    img -= 1
    return img

image_list = glob(images_path+'/*.jpg')
print(len(image_list),' images Found in the folder')

image_dataset = tf.data.Dataset.from_tensor_slices(image_list)
image_dataset = image_dataset.shuffle(buffer_size=10240)
image_dataset = image_dataset.map(data_load, num_parallel_calls=AUTOTUNE)
image_dataset = image_dataset.batch(batch_size)
image_dataset = image_dataset.prefetch(buffer_size=AUTOTUNE)

model_folders = "model_files" #@param {type:"string"}
generator_weights = "no_model" #@param {type:"string"}
discriminator_weights = "no_model" #@param {type:"string"}



Using TensorFlow backend.


52  images Found in the folder


#Making the Model -Image is needed-




In [ ]:
#@title Build The Model Dynamically Based on The Image Dimensions and Training the model

import tensorflow as tf


from tensorflow.keras.layers import Activation,Reshape,Input,Dense,LeakyReLU,Conv2D,ReLU,BatchNormalization,Conv2DTranspose
from tensorflow.keras import Model
from tensorflow.random import normal

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow import ones_like,zeros_like,GradientTape,clip_by_value
from tensorflow.summary import scalar

def deconv_block(input_tensor, num_filters, kernel_size, strides, bn=True):
    x = Conv2DTranspose(filters=num_filters, kernel_initializer=w_init, kernel_size=kernel_size, padding='same', strides=strides, use_bias=False if bn else True)(input_tensor)
    if bn:
        x = BatchNormalization()(x)
        x = ReLU()(x)
    return x


def conv_block(input_tensor, num_filters, kernel_size, padding='same', strides=2, bn=True, activation=True):
    x = Conv2D(filters=num_filters, kernel_initializer=w_init, kernel_size=kernel_size, padding=padding, strides=strides, use_bias=False if bn else True)(input_tensor)
    if bn:
        x = BatchNormalization()(x)
    if activation:
        x = LeakyReLU(alpha=0.2)(x)
    return x

def build_generator(latent_dim=100):
    f = [2**i for i in range(5)][::-1]
    noise = Input(shape=(latent_dim,), name='generator_noise_input')
    x = Dense(f[0] * filters * height_output * width_output)(noise)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Reshape(target_shape=[height_output, width_output, 16 * filters])(x)
    for i in range(1, 5):
        x = deconv_block(x,num_filters=f[i] * filters,kernel_size=5,strides=2,bn=True)
    x = deconv_block(x, num_filters=3,kernel_size=3,strides=1,bn=False)
    fake_output = Activation('tanh', name='generator_output')(x)
    return Model(inputs=[noise],outputs=[fake_output],name='Generator')


def build_discriminator():
    image_input = Input(shape=[image_height, image_width, 3], name='discriminator_image_input')
    f = [2**i for i in range(4)]
    x = conv_block(image_input, num_filters=f[0] * filters, kernel_size=5, strides=2, bn=False)
    for i in range(1, 4):
        x = conv_block(x, num_filters=f[i] * filters, kernel_size=5, strides=2, bn=True)
    x = conv_block(x, num_filters=1, kernel_size=height_output, padding='valid', strides=1, bn=False, activation=False)
    classification_logits = Reshape(target_shape=[1])(x)
    return Model(inputs=[image_input], outputs=[classification_logits], name='Discriminator')

generator = build_generator(latent_dim)
generator.summary()

discriminator = build_discriminator()
discriminator.summary()

if not (generator_weights == "no_model" or discriminator_weights == "no_model") : 
  generator.load_weights(os.path.join(model_path,generator_weights))
  discriminator.load_weights(os.path.join(model_path, discriminator_weights))


bce_loss_fn = BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
discriminator_optimizer = Adam(learning_rate=0.00025, beta_1=0.5)
generator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)


def loss_D(real_logits, fake_logits):
    real_loss = 0.5 * bce_loss_fn(ones_like(real_logits), real_logits)
    fake_loss = 0.5 * bce_loss_fn(zeros_like(fake_logits), fake_logits)
    return real_loss + fake_loss


def loss_G(fake_logits):
    loss = bce_loss_fn(ones_like(fake_logits), fake_logits)
    return loss


def loss_D_real(real_logits):
    real_loss = bce_loss_fn(ones_like(real_logits), real_logits)
    return real_loss


def loss_D_fake(fake_logits):
    fake_loss = bce_loss_fn(zeros_like(fake_logits), fake_logits)
    return fake_loss  

@tf.function
def training_step(images):
    noise = normal(shape=[batch_size, latent_dim], mean=0, stddev=1)

    # calculate the real discriminator loss 
    with tf.GradientTape() as real_tape:
        discriminator_loss_real = loss_D_real(discriminator(images, training=True))
        discriminator_gradients_real = real_tape.gradient(discriminator_loss_real, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients_real, discriminator.trainable_variables))

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_logits = discriminator(generator(noise, training=True), training=True)

        discriminator_loss_fake = loss_D_fake(fake_logits)
        generator_loss = loss_G(fake_logits)

        discriminator_gradients_fake = disc_tape.gradient(discriminator_loss_fake, discriminator.trainable_variables)
        generator_gradients = gen_tape.gradient(generator_loss, generator.trainable_variables)

        discriminator_optimizer.apply_gradients(zip(discriminator_gradients_fake, discriminator.trainable_variables))
        generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))

    return generator_loss, discriminator_loss_real, discriminator_loss_fake

def save_generated_images(noise, epoch=None):
    images = generator(noise)
    images = tf.clip_by_value((images + 1) * 127.5, 0, 255).numpy()
    if not os.path.isdir("training_images"):
      os.mkdir("training_images")     
    for i in range(16):
        cv2.imwrite(f'training_images/image_number_{i}_ep_{epoch}.png',cv2.cvtColor(images[i], cv2.COLOR_RGB2BGR))        

def train(epochs=30, save_every=3, steps=None):
    batch_losses = {'g_loss': [], 'd_loss': []}
    epoch_losses = {'g_loss': [], 'd_loss': []}
    for ep in range(epochs):
        running_loss = {'g_loss': [], 'd_loss': []}
        for step, images in enumerate(image_dataset):
            batch_g_loss, batch_d_loss_real, batch_d_loss_fake = training_step(images)
            batch_d_loss = batch_d_loss_real + batch_d_loss_fake
            running_loss['g_loss'].append(batch_g_loss.numpy())
            running_loss['d_loss'].append(batch_d_loss.numpy())
        print(f'||epoch {ep+1}/{epochs} step {step+1}/{steps}|G_LOSS : {batch_g_loss:.3f}|D_LOSS : {batch_d_loss:.3f}||')    
        batch_losses['g_loss'].extend(running_loss['g_loss'])
        batch_losses['d_loss'].extend(running_loss['d_loss'])
        epoch_losses['g_loss'].append(np.mean(running_loss['g_loss']))
        epoch_losses['d_loss'].append(np.mean(running_loss['d_loss']))
        if (ep + 1) % save_every == 0:
            print(f'||saving weights for epoch : {ep+1}||')
            generator.save_weights(os.path.join(model_folders,f'generator_weights_{ep+1}.h5'))
            discriminator.save_weights(os.path.join(model_folders,f'discriminator_weights_{ep+1}.h5'))
            save_generated_images(display_noise, epoch=f'{ep+1}_{step+1}')
    return batch_losses, epoch_losses

total_epochs = 10 #@param {type:"integer"}
save_every = 1 #@param {type:"integer"}

batch_losses, epoch_losses = train(total_epochs, save_every, len(image_list)//batch_size) 


# Notes : 

### *save the colab file in your drive or in playground mode 
#### *for the augmentaion make sure you select the augmentaion flag then add augmentaion folder name 
### *This code is modified version of : [TFDCGAN](https://www.tensorflow.org/tutorials/generative/dcgan)
### *learn more tips for DCGAN [Link](https://medium.com/intel-student-ambassadors/tips-on-training-your-gans-faster-and-achieve-better-results-9200354acaa5)
### *For image height and width Have to be x^2
 
